# 這堂課的任務
  * Step 0: 引入會用到的套件
  * Step 1: 下載語料檔 (語料已經過斷詞前處理)
  * Step 2: 將語料讀進來
  * Step 3: 觀察語料
  * Step 4: 對語料進行 Indexing
  * Step 5: 實作 Concordance
  * Step 6: 實作 Collocation
  
  > 迷路時的提醒：請記得真實世界中的例子，你現在手上有一本小說 (Step 0 - 3)，小說的最後還沒有任何Index，所以我們首先要進行 Indexing (Step 4)，產生出 Index 後 ，我們要利用這個 Index 做查詢 (Step 5, 6)。

# Step 0: 引入會用到的套件

In [ ]:
# 用來讀取以json格式儲存的語料檔 [Step 2]
import json

# defaultdict 用來方便進行 indexing
# Counter 用來方便統計各版文章數
# [Step 3, 4]
from collections import defaultdict, Counter 

# 用來計算程式執行時間 [Step 5]
import time

# 用來對 token 做 regular expression 查詢 [Step 5]
import re

# 用來顯示美美的查詢結果 [Step 5]
import tabulate
import pandas as pd
from IPython.display import HTML, display

%load_ext google.colab.data_table

# Step 1: 下載語料檔

In [ ]:
# 用來下載 Google Drive 文件的 Python 套件
# Colab 已內建此指令
!pip install gdown

In [ ]:
!gdown --id "1mmAdISiVBw_acDRiyoAFFOIJoyDQ6dPa" -O "dcard2.jsonl"

In [ ]:
# TODO: 看 dcard2.jsonl 的前五行


<a href="https://drive.google.com/file/d/1Yp4501Db7Dyj_gJg0f107FnTDg2ohhhw/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1Yp4501Db7Dyj_gJg0f107FnTDg2ohhhw" width="400"></a>

觀察：我們的語料中，有什麼 metadata 和 data?

# Step 2: 將語料讀進來

In [ ]:
RAW_DATA_PATH = "dcard2.jsonl"

# 用一個空的 list 準備裝進語料
corpus = []

# TODO: 將每行 json 一一讀進 corpus


<a href="https://drive.google.com/file/d/1KR9XvKZS1sgGtS_Eqo91-vAUOdDrkh5W/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1KR9XvKZS1sgGtS_Eqo91-vAUOdDrkh5W" width="900"></a>


# Step 3: 觀察語料

## 3-0: 認識語料的資料結構

In [ ]:
# TODO: 查看總文章篇數
# 因為 corpus 是一個 list，我們剛剛把每篇文章一一加入這個 list
# 所以用 len() 去讀取這個 list 的長度，就是去看我們總共有幾篇文章


In [ ]:
# TODO: 查看語料中的第零篇文章內容


In [ ]:
# TODO: 查看第零篇文章的版名


In [ ]:
# TODO: 查看第零篇文章中的第零個句子


In [ ]:
# TODO: 查看第零篇文章中的第零個句子中的第零個token


In [ ]:
# TODO: 查看第零篇文章中的第零篇句子中的第零篇token的詞目是什麼


<a href="https://drive.google.com/file/d/1SCPNqwhftuRLHI7jd-l7xEc620kZRjNC/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1SCPNqwhftuRLHI7jd-l7xEc620kZRjNC" width="900"></a>

`corpus` 是我們用來儲存語料的地方，由於我們可以餵位置給`corpus`，讓`corpus`告訴我們該位置的詞是什麼。在這個意義下，`corpus`其實就是一個 Foward Index (正向索引)。

## 3-1: 觀察 metadata

In [ ]:
# TODO: 看我們的語料中到底有哪些版，以及每個版各有幾篇文章
#       請讓 forum 這個變數的內容最後的結果如下圖
# 我們將以 collections.Counter 來達成這個目標
# 使用前要先引入: from collections import Counter
forum = Counter()

# 請接著完成版名的統計


<a href="https://drive.google.com/file/d/17WIpNn8m_7UPyjHXOSlJ331v51Jtknzm/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=17WIpNn8m_7UPyjHXOSlJ331v51Jtknzm" width="400"></a>

In [ ]:
# TODO: 看一下 forum 長什麼樣子


In [ ]:
# TODO: 叫出前十多文章的版
# 請使用 Counter.most_common()


# Step 4: 對語料進行 Indexing (重頭戲🎪)

想像你是一個機器人，現在我丟給你一本斷好詞的「哈利波特：阿茲卡班的逃犯」，

請你幫我產生一個可以附在最後面的索引，好讓讀者可以去查「妙麗」出現的地方。

這時你會怎麼做？

## 4-0: 暖身操: 利用 `enumerate()` 取得list中的位置訊息

In [ ]:
s = ["今天", "天氣", "晴"]

for element in s:
    print(element)

In [ ]:
# TODO: 請使用 enumerate() 在印出 list 當中的每個元素的同時，也一併印出該元素的 index
# 範例輸出：
# 0 今天
# 1 天氣
# 2 晴

In [ ]:
# 假設我們今天有一個更複雜的語料結構（與我們的語料結構雷同）

fake_corpus = [
    # article 0
    [
        # setence 0
        [
            {"word": "這", "pos": "NH"},     # token 0
            {"word": "是", "pos": "SHI"},    # token 1
            {"word": "第", "pos": "N"},      # token 2
            {"word": "零", "pos": "X"},      # ...
            {"word": "句", "pos": "X"},
         
        ],
        # sentence 1
        [
            {"word": "這", "pos": "NH"},
            {"word": "是", "pos": "SHI"},
            {"word": "第", "pos": "N"},
            {"word": "一", "pos": "X"},
            {"word": "句", "pos": "X"},
        ],
    ],

    # article 1
    [
        # sentence 0
        [
            {"word": "另", "pos": "NH"},
            {"word": "一", "pos": "SHI"},
            {"word": "篇", "pos": "N"},
            {"word": "文章", "pos": "X"},
        ],
    ]
]

In [ ]:
# TODO: 請用迴圈取出每一個token的詞目(word)，讓輸出的每行是一個單獨的token詞目
# 範例輸出:
# 這
# 是
# 第
# 零
# 句
# 這
# 是
# 第
# 一
# 句
# 另
# 一
# 篇
# 文章


In [ ]:
# TODO: 那要如何搭配 enumerate() 才能給出每一個詞目所在的 文章id, 句子id, token id 呢？
# 範例輸出:
# 0 0 0 這
# 0 0 1 是
# 0 0 2 第
# 0 0 3 零
# 0 0 4 句
# 0 1 0 這
# 0 1 1 是
# 0 1 2 第
# 0 1 3 一
# 0 1 4 句
# 1 0 0 另
# 1 0 1 一
# 1 0 2 篇
# 1 0 3 文章


## 4-1: 對詞目進行Indexing

In [ ]:
word_index = {}    # 用來儲存詞目的索引

# TODO: 請用 4-0 所提到的方法，對 corpus 做讀取，並讓 word_index 最後的結果如下圖


<a href="https://drive.google.com/file/d/1a7bs6Pux1CINo1ONGf_C0DW6aRXSgSkO/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1a7bs6Pux1CINo1ONGf_C0DW6aRXSgSkO" width="400"></a>

## 4-2: 換個方式對詞性做Indexing

使用 Python 提供給我們的捷徑: `defaultdict`

要先 import 進來才能使用:
```python
from collections import defaultdict
```

In [ ]:
pos_index = defaultdict(list)     # 用來儲存詞性的索引

# 對 dcard2.jsonl 中的每一行進行讀取。每一行就是一篇貼文。
for article_idx, article in enumerate(corpus):
    for sentence_idx, sentence in enumerate(article['text']):
        for token_idx, token in enumerate(sentence):
            
            locator = [article_idx, sentence_idx, token_idx]

            # 使用 defaultdict 的方便之處在於
            # 這裡我們就不用多做 4-1 中所做的判斷
            # defaultdict 會自動幫我們處理這件事
            pos_index[token['pos']].append(locator) 

## 4-3: 查看 Indexing 後的結果

In [ ]:
# TODO: 叫出「虐心」這個詞在我們語料中所有出現的地方
# 並解讀出現結果的結構/意義是什麼


<a href="https://drive.google.com/file/d/1nZ-h8McCSXVrHKqz-hro_dZMjoMefRn5/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1nZ-h8McCSXVrHKqz-hro_dZMjoMefRn5" width="400"></a>

In [ ]:
#TODO: 寫一個函數 word_freq(word)
#      輸入一個詞，回傳該詞詞頻
def word_freq(word):
    pass

In [ ]:
# TODO: 請找出「的」的詞頻


In [ ]:
#TODO: 寫一個函數 pos_freq(pos)
#      輸入一個詞性，回傳該詞性頻率
def pos_freq(pos):
    pass

In [ ]:
# TODO: 請找出我們語料中word type的數量


In [ ]:
# TODO: 請找出我們語料中pos type的數量


In [ ]:
# TODO: 請列出所有的詞性
token_sum = 0

for word, list_of_locator in word_index.items():
    token_sum += word_freq(word)

詞性意義請參考：
- [中研院平衡語料庫詞類標記集](http://ckipsvr.iis.sinica.edu.tw/papers/category_list.pdf)
- [中研院詞庫小組. (1993). 中文詞類分析(三版)](https://ckip.iis.sinica.edu.tw/data/paper/report/ckip-9305.pdf)

In [ ]:
# TODO: 請找出我們語料的總token數是多少


## [插曲] 比較有無 index 的時間差

In [ ]:
def compare_time_with_and_without_index(word):

    print(f"查詢的詞: {word}")
    print(f"該詞於語料庫中的詞頻: {len(word_index[word])}")


    ###### 使用 index: 給時間寶貴的大忙人 ######
    t1 = time.time()
    for a_idx, s_idx, w_idx in word_index[word]:
        pass
    t2 = time.time()
    t_with_index = t2 - t1
    

    ###### 不使用 index: 給勤奮的螞蟻 ######
    t1 = time.time()
    
    # TODO: 如果要每次搜尋都要重新掃過一整個語料庫看某詞有沒有出現
    pass
    
    t2 = time.time()
    t_without_index = t2 - t1

    ###### 不使用index的時間 比上 使用index的時間 ######
    ratio = t_without_index / t_with_index
    
    # 印出結果
    print(f"使用index的時間: {t_with_index:.6f}")
    print(f"不使用index的時間: {t_without_index:.6f}")
    print(f"without_index / with_index = {ratio:6f}")
    print("-----------")

In [ ]:
compare_time_with_and_without_index("虐心")

In [ ]:
# TODO: 試看看針對不同出現頻率的詞，使用/不使用index的時間差距，並試著解讀為什麼會有這樣的差別。
# 比較: 出現 1      次的 "珍妮"
#      出現 510    次的 "造成"
#      出現 87610  次的 "是" 
#      出現 231504 次的 "的"


為什麼一詞的詞頻越大，使用 index 的效率會越來越接近不使用 index 的效率？ 感受一下，這樣的差距與日常生活中的經驗是否一致？

# Step 5: Concordance



## 5-0: 從 word_index 中搜尋單詞出現的句子

In [ ]:
# TODO: 請利用 word_index ，印出所有出現「虐心」一詞的句子
#       每個詞目之間請以一個空格隔開


## 5-1: 加進 window_size 的考量

In [ ]:
# 先以其中一個「虐心」出現的地方為例: [5777, 7, 4]

# TODO: 宣告一個變數 sentence，並將 sentence 設為語料中的第 5777 篇文章中的第 7 句
sentence = 

# sentence 是一個 list of tokens
print(sentence)    

In [ ]:
# TODO: 請列出此句中的每個詞目為何
#       每個詞目之間請以一個空格隔開


In [ ]:
# TODO: 請印出此句中「虐心」的前 3 個詞到後 3 個詞 (window_size 為 3)
window_size = 3



<a href="https://drive.google.com/file/d/1RGJzG7SHH_aBZvz0FDK1KBcFwKnSRj4l/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1RGJzG7SHH_aBZvz0FDK1KBcFwKnSRj4l" width="400"></a>

In [ ]:
# TODO: 將 window_size 設為 5 看看

window_size = 5



In [ ]:
# 錯誤來自於兩個地方
#    (1) (該詞出現的位置 - window_size) 可能會小於 0: 也就是會超出句子左邊的邊界
#    (2) (該詞出現的位置 + window_size) 可能會大於 句子長度: 也就是會超出句子右邊的邊界
# 所以我們必須設停損點:
#    (1) 對左邊邊界而言，如果 (該詞出現的位置 - window_size) < 0 ，那麼就停在 0 數值 => 使用 max(0, (該詞出現的位置 - window_size))
#    (2) 對右邊邊界而言，如果 (該詞出現的位置 + window_size) > 句子長度 ，那麼就停在句子長度的數值 => min(句子長度, (該詞出現的位置 + window_size))

# TODO: 讓我們來加進安全邊界的考量吧

window_size = 100

safe_left_bound = pass
safe_right_bound = pass

for i in range(safe_left_bound, safe_right_bound):
  print(sentence[i]["word"] + " ", end="")

## 5-2 完成第一個 `query_a_word()` 功能

能查詢單個詞目，並按照需求顯示相關資訊。

執行範例：
```python
query_a_word("虐心", window_size=5, show_pos=False, metadata=["forumName", "likeCount"])
```

會回傳
```python
[ 
     {"article_id": 30, "left": "這部戲真的很", "keyword": "虐心", "right": "！", "forumName": "戲劇", "likeCount": 20 }, # 第0句
     {"article_id": 33, "left": "真的太", "keyword": "虐心", "right": "了吧。", "forumName": "文學", "likeCount": 20 }, # 第1句
     ...
 ]
```

In [ ]:
# TODO: 寫一個函式 query_a_word(word, window_size, show_pos), 接收三個參數:
#     word: 要搜尋的關鍵字 <字串>
#     window_size: 指定視窗大小 <整數>
#     show_pos: 是否顯示詞性 <Bool>
# 此函數將回傳所有該關鍵字出現的句子，結構為 list of dicts

def query_a_word(word, window_size=5, show_pos=False, metadata=[]):

    # 最後的結果
    results = []

    # 從索引中叫出該詞出現的地方，並用迴圈一一處理
    

        # 該詞出現的某個句子


        # 準備好每個要裝進 results 的元素
        each_concordance_line = {
            
        }

        # 需要哪些metadata，就把那些metadata裝進來


        # 判斷關鍵字是否要顯示詞性


        # 安全邊界


        # 先處理處理關鍵字的左邊部分


        # 再處理處理關鍵字的右邊部分
 


        # 每次處理完後記得把 each_concordance_line 加進 results 中
        results.append(each_concordance_line)

    return results

In [ ]:
# TODO: 試著使用 query_a_word() 看看


In [ ]:
# TODO: 把結果餵給 pd.DataFrame()
columns_order = ['article_id', 'forumName', 'likeCount', "left", "keyword", "right"]



## 5-3: 重新組織一下巨大的 `query_a_word()`

<a href="https://drive.google.com/file/d/1qpiJjz6oQOF1w0tMg0O4Wk5QSVvW3VWg/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1qpiJjz6oQOF1w0tMg0O4Wk5QSVvW3VWg" width="400"></a>

`query_a_word()` 裡頭做的事，其實可以分成兩大部分:

1. 篩選index: 從 index 中篩選出符合某一個 query 條件的 locator
 - 詞目為 "虐心"
 - 詞性為 "V"
 - 多詞目並列： "做" "一" "個"
 - 詞性, 詞目並列： "V" "起來"
2. 生成一個個的concordance line: 處理顯示結果
 - 需不需要顯示詞性？
 - 要給出幾個 window_size 的上下文?
 - 需要給出哪些metadata?

區分這兩大部分的好處，有助於我們之後做更複雜的查詢時，可以更專注地處理程式的不同部分。

In [ ]:
def query_a_word(word, window_size=5, show_pos=False, metadata=[]):

    results = []

    # 在這裡我們用 locator 取代原本的 a_idx, s_idx, t_idx
    # 所以現在 locator 就會是一個 list: [a_idx, s_idx, t_idx]
    # 我們把開箱 locator 的任務交給 generate_concordance_line()
    for locator in filter_a_word(word):       
        result = generate_concordance_line(locator, window_size, show_pos, metadata)
        results.append(result)

    return results


def filter_a_word(word):
    return word_index[word]


def generate_concordance_line(locator, window_size, show_pos, metadata):
    a_idx = locator[0]
    s_idx = locator[1]
    t_idx = locator[2]

    sentence = corpus[a_idx]['text'][s_idx]

    # 準備好每個元素
    each_concordance_line = {
        "article_id": a_idx,
        "left": "",
        "keyword": "",
        "right": ""
    }

    # 需要哪些metadata，就把那些metadata裝進來
    for key in metadata:
        each_concordance_line[key] = corpus[a_idx][key]

    # 判斷是否要顯示詞性
    if show_pos:
        each_concordance_line["keyword"] = sentence[t_idx]["word"] + "/" + sentence[t_idx]["pos"]
    else:
        each_concordance_line["keyword"] = sentence[t_idx]["word"]

    # 安全邊界
    safe_left_bound = max(0, t_idx - window_size)
    safe_right_bound = min(len(sentence), t_idx + window_size + 1)


    # 先處理處理關鍵字的左邊部分
    for token in sentence[safe_left_bound: t_idx]:
        if show_pos:
            each_concordance_line["left"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["left"] += token["word"]

    # 再處理處理關鍵字的右邊部分
    for token in sentence[t_idx + 1: safe_right_bound]:
        if show_pos:
            each_concordance_line["right"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["right"] += token["word"]

    return each_concordance_line


## 5-4: 美美的資料，閱讀舒適度提升

1. 資料與資料呈現
2.  閱讀版面舒適度調整

<a href="https://drive.google.com/file/d/1sNPMBGdCMGTiqKunsYpAw0r3BdhQQalL/view" target="_blank"><img src="https://drive.google.com/u/0/uc?id=1sNPMBGdCMGTiqKunsYpAw0r3BdhQQalL&export=download" width="85%"></a>

> 關於資料呈現的想像：直接面對海量資料就好像近視沒戴眼鏡，或是去看 3D 電影不帶特製眼鏡一樣，好像有東西在那裡卻又好模糊、不確知那是什麼或是傳達了什麼。

資料視覺化的工具 ( [Charting in Colaboratory](https://colab.research.google.com/notebooks/charts.ipynb#scrollTo=QSMmdrrVLZ-N)  )

- [Matplotlib](https://matplotlib.org/) : the most common charting package. 
- [Seaborn](http://seaborn.pydata.org/) : One of several libraries layered on top of Matplotlib that is worth highlighting, and you can use in Colab. (Colaboratory charts use Seaborn's custom styling by default.)
- [Altair](https://altair-viz.github.io/) : a declarative visualization library for creating interactive visualizations in Python, and is installed and enabled in Colab by default. 


### 5-4-0: 若是「文字資料」的呈現呢？

<img src="https://drive.google.com/u/0/uc?id=1KultyJIj4cGq6E9LhF0tHLTRo_7073E3&export=download" width="50%">


> 適合一直放在心上的探問：要怎傳達(資料)會是最有效、直覺的？

### 5-4-1: [python-tabulate](https://pypi.org/project/tabulate/) 

- 輕鬆印出表格：只要呼叫一個 function 就能完成！
- 將輕量純文本轉為表格數據：具多種輸出格式供後續編輯與轉換
- 混合的文本與數據資料也能很好讀：欄位對齊、數字格式


In [ ]:
# 感覺一下使用 python-tabulate 呈現資料
from tabulate import tabulate

In [ ]:
table = [["Sun",696000,1989100000],["Earth",6371,5973.6],["Moon",1737,73.5],["Mars",3390,641.85]]

print(table)
print(tabulate(table, headers=["Planet","R (km)", "mass (x 10^29 kg)"]))

In [ ]:
print(tabulate(result_nuexin, headers="keys", showindex="always", tablefmt="simple"))

當資料不只有 10 筆時，看起來會怎麼樣呢？

In [ ]:
result_yen = query_a_word("語言", 10)

print(tabulate(result_yen, headers="keys", showindex="always", tablefmt="simple"))

### 5-4-2: [pandas](https://pandas.pydata.org/docs/user_guide/index.html)

- 讀取表格數據(tabular data)資料的超實用工具
- 資料類型：Series(一維) ＆ DataFrame(二維, 很常用)
- 直接看 [Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) 了解更多


In [ ]:
# 使用 pandas 中的 pd.DataFrame() 的結果
import pandas as pd

pd.DataFrame(result_nuexin)

In [ ]:
#TODO: 請大家使用 query_a_word() 查詢「語言」，並把結果餵給 pd.DataFrame()


In [ ]:
pd.DataFrame(query_a_word("是", 10), columns=columns_order)

> 資料筆數多（87610 筆），但是看不到全部 >>> 有更好的呈現方式嗎？互動的可能性？

### 5-4-3: [Data Table Display](https://colab.research.google.com/notebooks/data_table.ipynb)  

- 互動性呈現：篩選、分類、分頁設定等
- 可客製化

In [ ]:
# 加入 Data Table Display extension
# renders pandas dataframes into interactive displays that can be filtered, sorted, and explored dynamically

%load_ext google.colab.data_table

In [ ]:
pd.DataFrame(result_nuexin)

#TODO: 選出按讚數為 50~100 之間的資料

#TODO: 找出包含「...」、「》」、「〖 〗」的資料


In [ ]:
# 直接生成 data tables 
from google.colab import data_table

# 可使用參數達到更多客製化的呈現
data_table.DataTable(pass)

### 5-4-4: 天哪，我把資料變好看了！ 

#### 玩美步驟

1. 在網頁上按右鍵 
2. 選擇「檢查」 來看 html 程式碼 
3. 找到我們的目標區段（css selector）
4. 寫 css code 來改變「對齊方式」、「顏色」
5. 資料更好看了！（得到美美的 corpus 資料呈現格式）

html 圖解

<a href="https://drive.google.com/file/d/1IeTnQ8yRptYu60Naaj7iZeBbJq-eOd0o/view?usp=sharing" target="_blank"><img src="https://drive.google.com/u/0/uc?id=1IeTnQ8yRptYu60Naaj7iZeBbJq-eOd0o&export=download" width="80%"></a>

In [ ]:
# TODO: 利用 html, css selector 來改變資料呈現 (ref. slide page 8)

from IPython.display import HTML, display

def set_css_in_cell_output():
  display(HTML("""

  """))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)

In [ ]:
pd.DataFrame(result_nuexin)

In [ ]:
# TODO: 自己動手玩看看～
#        1. 讀取真實的範例資料 (火大-16, 全家-72, 有事-97, 難過-541)
#        2. 試試看讓每一頁呈現五筆資料就好
#        3. 改變 html 呈現：變更 keyword 欄位的顏色、字型或字體大小；調整所有欄位的對齊方式為置中。



## 5-5: 要查詢單詞，也要查詢單個詞性

請試著根據 5-3 所寫的 `query_a_word()`，寫一個 `query_a_token()`，讓這個函數既能查詢單一個word，也能查詢單一個pos。

執行範例：
```python
# 找出所有詞目為「虐心」的token
query_a_token(token_value="虐心", token_type="word", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])

# 找出所有詞性為「VA」的token
query_a_token(token_value="VA", token_type="pos", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
```

In [ ]:
# 注意: query_a_token() 跟剛才的 query_a_word() 只有兩個不同的小地方
#      1. query_a_token() 的前兩個參數變成 token_value 和 token_type
#      2. filter_a_word(word) 變成了 filter_a_token(token_value, token_type)
# TODO: 請大家完成 filter_a_token(token_value, token_type) 的內容

def query_a_token(token_value, token_type, window_size=5, show_pos=False, metadata=[]):

    results = []

    for locator in filter_a_token(token_value, token_type):
        result = generate_concordance_line(locator, window_size, show_pos, metadata)
        results.append(result)

    return results


def filter_a_token(token_value, token_type):

    # 提示: 先判斷 token_type 為何，才知道要使用哪個 index
    pass

In [ ]:
# TODO: 自由時間，填入你想查的token吧，也嘗試看看修改不同參數的結果


## 5-6: 在查詢單個token的基礎上，加一點regular expression

請試著根據 5-5 所寫的 `query_a_token()`，寫一個 `query_a_regex_token()`，讓我們可以用 regular expression 查詢單一個word，也能查詢單一個pos。

用處：
- 找出所有以「者」為後綴的詞。
- 找出所有以「老」為前綴的詞。
- 找出所有動詞性的詞。

執行範例：
```python
# 找出所有以「者」為後綴的詞
query_a_regex_token(token_value=".+者$", token_type="word", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])

# 找出所有以「老」為前綴的詞
query_a_regex_token(token_value="^老.+", token_type="word", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])

# 找出所有被標為動詞的詞
query_a_regex_token(token_value="^V.+", token_type="pos", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
```

In [ ]:
# 注意: query_a_regex_token() 跟剛才的 query_a_token() 只有一個不同點
#      1. filter_a_token(token_value, token_type) 變成了 filter_a_regex_token(token_value, token_type)
# TODO: 請大家完成 filter_a_regex_token(token_value, token_type) 的內容

import re

def query_a_regex_token(token_value, token_type, window_size=5, show_pos=False, metadata=[]):

    results = []

    for locator in filter_a_regex_token(token_value, token_type):
        result = generate_concordance_line(locator, window_size, show_pos, metadata)
        results.append(result)

    return results


def filter_a_regex_token(token_value, token_type):
    pass



In [ ]:
# TODO: 試著找看看所有以「者」為後綴的token


In [ ]:
# TODO: 找出所有以「老」開頭的token


## 5-7: 限定某詞性下的某詞 (回家練習)

在中文當中，一個詞目可能會有不同的詞性，像是「花」錢和茉莉「花」。請寫一個函數，限定某個詞性下的詞。

## 5-8: [進階] 搜尋多個詞並列的 Phrase query

目前前面所實作的 `query_a_word()`, `query_a_token()` 和 `query_a_regex_token()` 都只能搜索單一個token。

在搜尋語料時，我們可能還會需要針對多個並列的 token 進行搜索，例如 `query_phrase(["做", "一", "個"])`。

所以我們現在要開始實作多個token並列的情況。

執行範例：
```python
# 找出所有 「做」「一」「個」的句子

phrase = [
    {"type": "word", "value": "做"},
    {"type": "word", "value": "一"},
    {"type": "word", "value": "個"},
]

query_phrase(tokens=phrase, window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
```

In [ ]:
def query_phrase(tokens, window_size=5, show_pos=False, metadata=[]):

    results = []

    for locator in filter_phrase(tokens):
        result = generate_concordance_line_v2(locator, window_size, show_pos, metadata)
        results.append(result)

    return results

In [ ]:
def filter_phrase(tokens):

    results = []
    token_freqs = []

    # 先比較傳進來的 list of tokens，哪一個 token 的出現次數最少 => 為了對搜尋做優化
    for token in tokens:
        if token["type"] == "word":
            # 使用我們前面已經寫好的 word_freq 函數
            token_freq = word_freq(token["value"])

        elif token["type"] == "pos":
            token_freq = 1000000000
            # token_freq = sum([pos_freq(pos) for pos in pos_index.keys() if re.match(token["value"], pos)])
            # len(pos_index[token["value"]])
        else:
            raise ValueError("token type只能是 word 或 pos")

        token_freqs.append(token_freq)

    # 找出 token_freqs 最小的
    min_freq = min(token_freqs)

    # 找出出現次數最少的 token 是的第幾個
    min_freq_idx = token_freqs.index(min_freq)

    # 找出出現次數最少的 token value
    min_freq_token_value = tokens[min_freq_idx]["value"]
    
    # 出現次數最少的 token 的位置離tokens 最左邊的元素多遠?
    normalized_index = [i - min_freq_idx for i in range(len(tokens))]

    # 從出現次數最少次的 token 開始找
    for a_idx, s_idx, t_idx in word_index[min_freq_token_value]:
        
        need_to_pass = False
        sentence = corpus[a_idx]["text"][s_idx]

        for i in range(len(tokens)):
            if i != min_freq_idx:
                # 因為可能會超過句子邊界，所以使用 try...except...處理這種情況
                try:
                    if tokens[i]["type"] == "word":
                        if sentence[t_idx + (i - min_freq_idx)][tokens[i]["type"]] != tokens[i]["value"]:
                            need_to_pass = True
                            break
                    elif tokens[i]["type"] == "pos":
                        if not re.match(tokens[i]["value"], sentence[t_idx + (i - min_freq_idx)][tokens[i]["type"]]):
                            need_to_pass = True
                            break

                except:
                    need_to_pass = True
                    break
        if need_to_pass:
            continue

        results.append([a_idx, s_idx, [i + t_idx for i in normalized_index]])
    
    return results

In [ ]:
# 現在 locator 傳進來的東西是 [10, 2, [20, 21, 22]]
# 也就是說 t_idx 不再是單個整數，而是list of int
def generate_concordance_line_v2(locator, window_size, show_pos, metadata):
    a_idx = locator[0]
    s_idx = locator[1]
    t_idxs = locator[2]

    sentence = corpus[a_idx]['text'][s_idx]

    # 準備好每個元素
    each_concordance_line = {
        "article_id": a_idx,
        "left": "",
        "keyword": "",
        "right": ""
    }

    # 需要哪些metadata，就把那些metadata裝進來
    for key in metadata:
        each_concordance_line[key] = corpus[a_idx][key]

    # 判斷是否要顯示詞性
    if show_pos:
        each_concordance_line["keyword"] = " ".join([sentence[t_idx]["word"] + "/" + sentence[t_idx]["pos"] for t_idx in t_idxs])
    else:
        each_concordance_line["keyword"] = " ".join([sentence[t_idx]["word"] for t_idx in t_idxs])

    # 安全邊界
    safe_left_bound = max(0, t_idxs[0] - window_size)
    safe_right_bound = min(len(sentence), t_idxs[-1] + window_size + 1)


    # 先處理處理關鍵字的左邊部分
    for token in sentence[safe_left_bound: t_idxs[0]]:
        if show_pos:
            each_concordance_line["left"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["left"] += token["word"]

    # 再處理處理關鍵字的右邊部分
    for token in sentence[t_idxs[-1] + 1: safe_right_bound]:
        if show_pos:
            each_concordance_line["right"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["right"] += token["word"]

    return each_concordance_line

In [ ]:
# TODO: 請找出所有 "的" "動作" 的句子


## 5-9: 比較 `V+下來` 和 `V+下去`

In [ ]:
# TODO: 請找出所有 V + "下來" 的句子


In [ ]:
# TODO: 請找出所有 V + "下去" 的句子


In [ ]:
# TODO: 自由發揮看看


# Step 6: Collocation

![](https://static.cambridge.org/binary/version/id/urn:cambridge.org:id:binary:20180904072708260-0799:9781316410899:12570tbl3_1.png?pub-status=live)

![](https://static.cambridge.org/binary/version/id/urn:cambridge.org:id:binary:20180904072708260-0799:9781316410899:12570tbl3_2.png?pub-status=live)

各種Association measure:

![](https://static.cambridge.org/binary/version/id/urn:cambridge.org:id:binary:20180904072708260-0799:9781316410899:12570tbl3_3.png?pub-status=live)

## 6-1: 看「造成」一詞右邊一格的Collocation

右邊一格的意思就是 window size 為 1R (one 有時也能標為 +1)

在這個練習我們將以 LogDice 這個 association measure 為例子。

In [ ]:
# LogDice 算式中只出現 O11, R1, C1

## 以 造成 + 困擾 為例
### O11 為 造成 + 困擾 的總數
### R1  為 造成 + X    的總數 ~ 先以造成的總數來看
### C1  為 X   + 困擾  的總數 ~ 先以困擾的總數來看

# 先找出所有出現在「造成」後面一個的詞

result = {}

# 「造成」的詞頻
R1 = len(word_index['造成'])

for a_idx, s_idx, w_idx in word_index['造成']:
    
    sentence_with_造成 = corpus[a_idx]['text'][s_idx]

    # 跳過 造成 在句尾的情況
    if w_idx + 1 >= len(sentence_with_造成):
      continue

    # word_1r 為在這句話中，出現在「造成」後面的那個詞
    word_1r = corpus[a_idx]['text'][s_idx][w_idx + 1]['word']

    # 要是 word_1r 尚未出現在 result 之中，就初始化
    if word_1r not in result:
      result[word_1r] = {"O11": 1, "C1": len(word_index[word_1r]), "R1": R1}
    
    # 要是 word_1r 已經出現過了，那麼只要把他的 O11 加上 1 就好
    else:
      result[word_1r]["O11"] += 1

In [ ]:
result

In [ ]:
result_df = pd.DataFrame.from_dict(result, orient='index')

In [ ]:
#TODO: 寫一個運算 log_dice 的函數
import numpy as np
def log_dice(O11, R1, C1):
  return 14 + np.log(2 * O11 / (R1 + C1))

In [ ]:
result_df['log_dice'] = result_df.apply(lambda x: log_dice(x['O11'], x['R1'], x['C1']), axis=1)
result_df

In [ ]:
# TODO: 找自己有興趣的詞來看看